In [5]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
from googletrans import Translator
from google.transliteration import transliterate_word

In [6]:
filmlk_url = 'https://www.films.lk/'

In [7]:
filename = "artists-corpus/artists_links.csv"

def scape_main_pages():
    with open(filename,'a') as f:
        for i in range(1,10):
            main_url = 'https://www.films.lk/artists.php?page={}'.format(i)
            print("scrapping : " + main_url)
            response = requests.get(
                url = main_url,
            )

            page_soup = soup(response.content, 'html.parser')
            artists = page_soup.findAll("div",{"class":"column2"})


            for artist in artists:
                name = artist.span.text
                url = artist.a["href"]
                f.write(url + "\n")

            time.sleep(10)

In [8]:
scape_main_pages()

scrapping : https://www.films.lk/artists.php?page=1
scrapping : https://www.films.lk/artists.php?page=2
scrapping : https://www.films.lk/artists.php?page=3
scrapping : https://www.films.lk/artists.php?page=4
scrapping : https://www.films.lk/artists.php?page=5
scrapping : https://www.films.lk/artists.php?page=6
scrapping : https://www.films.lk/artists.php?page=7
scrapping : https://www.films.lk/artists.php?page=8
scrapping : https://www.films.lk/artists.php?page=9


In [14]:
def get_artist_list(): 
    with open(filename, "r") as f:
        artists_urls = f.readlines()
    
    artist_list = []
    
    for i in range(2):
        response = requests.get(
            url = filmlk_url + artists_urls[i].rstrip('\n'),
        )
  
        artist_meta_data = get_artist_data(response.content)            
                            
        artist_list.append(artist_meta_data)
        
        time.sleep(1)
        
    return artist_list            

In [ ]:
def get_artist_data(content):
    
    page_soup = soup(content, 'html.parser')
    artist = page_soup.findAll("div",{"class":"column2"})
    artist_summary = artist[1].findAll("li",{"class":None})
        
    artist_meta_data = {}

    for field in artist_summary:
        if field.b != None:

            if "Known As" in field.b.get_text():
                known_as_en = field.text.split(":")[1].strip()
                print("scrapping : " + known_as_en)
                artist_meta_data["known_as_en"] = known_as_en

            if "Sinhala" in field.b.get_text(): 
                known_as_si = field.text.split(":")[1].strip()
                artist_meta_data["known_as_si"] = known_as_si

            if "Real Name" in field.b.get_text(): 
                real_name = field.text.split(":")[1].strip()
                artist_meta_data["real_name"] = real_name

            if "Birthday" in field.b.get_text(): 
                birth = field.text.split(":")[1].strip() 
                artist_meta_data["birth"] = birth

            if "Died" in field.b.get_text(): 
                death = field.text.split(":")[1].strip()
                artist_meta_data["death"] = death
    
    return artist_meta_data
    

In [ ]:
artist_list = get_artist_list()
artist_list

scrapping : Rukmani Devi
scrapping : D.R. Nanayakkara
scrapping : B.A.W. Jayamanne


In [16]:
artist_list = get_artist_list()

with open ('artists-corpus/artists_data.json','w+') as f:
            f.write(json.dumps(artist_list))  

scrapping : Rukmani Devi
scrapping : D.R. Nanayakkara
scrapping : B.A.W. Jayamanne
scrapping : Eddy Jayamanna
scrapping : Clarice De Silva
scrapping : Sandya Kumari
scrapping : Prem Jayanth
scrapping : Ravindra Rupasena
scrapping : Gamini Fonseka
scrapping : Vijaya Kumaranatunga
scrapping : Joe Abeywickrama
scrapping : Dr. Lester James Peries
scrapping : Malani Fonseka
scrapping : Geetha Kumarasinghe
scrapping : Swarna Mallawarachchi
scrapping : Anoja Weerasinghe
scrapping : Freddie Silva
scrapping : Sanath Gunathilaka
scrapping : Sabeetha Perera
scrapping : Jeewan Kumaranatunga


In [ ]:
artist_info = page_soup.findAll("div",{"class":"column"})

In [ ]:

artist_meta_data["national_awards"] = []

for info in artist_info:
    field = info.find("span",{"class":"eventTitle"})   
    if (field.get_text() == 'NATIONAL AWARDS'):   
        na_td_rows = info.table.findAll("td",{"class":"row1"})
        for row in na_td_rows:
            if (row.span != None and row.a != None):
                national_award = {}
                award_title = row.span.text
                award_name = row.a.text
                
                national_award["award_title"] = award_title
                national_award["award_name"] = award_name
                
                
                
                artist_meta_data["national_awards"].append(national_award) 
                
                
print (artist_meta_data)               